<a href="https://colab.research.google.com/github/PingPingE/Learn_ML_DL/blob/main/Practice/Hands_On_ML/ch15-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd

# 긴 시퀀스 다루기
- 문제점 및 해결 방법(ch15-1에서 왜 tanh가 디폴트인지 알아보면서 살짝 다룸)
  - gradient 소실 또는 폭주 문제 -> 불안정한 학습
  - RNN이 긴 시퀀스를 처리할 때 입력의 첫 부분을 조금씩 잊어버리는 문제 

- RNN에 잘맞는 종류의 정규화: <strong>층 정규화</strong>(ch13-2에서 살짝 다룸)
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcdaNCJ%2FbtqAa7RVwhD%2FP60H1iai2Mxgvu09TGQpv0%2Fimg.png" width=80% height=80%/>

  - 즉, 배치 정규화는 배치 차원에 대해 정규화, 층 정규화는 <strong>특성 차원에 대해 정규화</strong>
  - 그래서 훈련과 테스트에서 동일한 방식으로 작동한다.
    - 배치 정규화는 훈련 세트의 모든 세트에 대한 특성 통계를 추정하기 위해 지수 이동 평균이 필요했음

## 메모리 셀 안에 층 정규화 구현

In [3]:
class LNSimpleRNNCell(keras.layers.Layer):
  def __init__(self, units, activation="tanh", **kwargs):
    super().__init__(**kwargs)
    self.state_size=units
    self.output_size=units
    self.simple_rnn_cell=keras.layers.SimpleRNNCell(units,activation=None)#===포인트: activation=None이다(활성화 함수에 넣기 전에 정규화하려고)
    self.layer_norm=keras.layers.LayerNormalization()
    self.activation=keras.activations.get(activation)
  
  def call(self, inputs, states):#====현재 타임 스텝의 inputs와 이전 타임 스텝의 hidden states(h_t-1)
    outputs, new_states= self.simple_rnn_cell(inputs, states)
    norm_outputs=self.activation(self.layer_norm(outputs)) #정규화하고 난 후의 값을 활성화 함수에 대입
    return norm_outputs, [norm_outputs] #===두 개인 이유: 하나는 출력, 하나는 새로운 은닉 상태(h_t)가 된다


- 사용자 정의 셀 적용

In [4]:
model=keras.models.Sequential([
                               keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True, input_shape=[None, 1]),
                               keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
                               keras.layers.TimeDistributed(keras.layers.Dense(10))
])

-----------
위 처럼 keras.layers.RNN층을 만들어서 LNSimpleRNNCell의 객체를 전달하면 된다.
- 만약 타임 스텝 사이에 dropout을 적용하고 싶다면?
  - 위처럼 드롭아웃을 적용하는 사용자 정의 셀을 만들어도 되긴하는데, 
  - keras.layers.SimpleRNN(recurrent_dropout=0.5) 매개변수를 지원함

## LSTM 셀